In [7]:
# Commented out IPython magic to ensure Python compatibility.
from google.colab import drive,userdata
import os

drive.mount('/content/drive', force_remount=True)
notebook_drive_path = "/content/drive/MyDrive/PyNucleus Project/Capstone Project V2.ipynb"
notebook_filename_in_repo = "Capstone Project.ipynb"
log_filename = "update_log.txt"

# GitHub details
GITHUB_USERNAME = userdata.get('GITHUB_USERNAME')
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
REPOSITORY_NAME = 'PyNucleus-Model'
REPO_PATH = f"/content/{REPOSITORY_NAME}" # The path where the repo will be cloned

# --- CLONE REPOSITORY OR SYNC IF IT EXISTS ---
if not os.path.exists(REPO_PATH):
    print(f"Cloning {REPOSITORY_NAME} repository...")
    repository_url = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPOSITORY_NAME}.git"
    !git clone {repository_url} {REPO_PATH}
    %cd {REPO_PATH}
else:
    print("Repository already exists. Syncing with GitHub...")
    %cd {REPO_PATH}
    # --- THIS IS THE CRUCIAL NEW STEP ---
    # Fetch the latest changes from GitHub and merge them into your local copy
    !git pull

# --- STEP 3: NAVIGATE INTO REPOSITORY & COPY YOUR NOTEBOOK ---

# The '%cd' command changes the current directory in Colab
%cd {REPO_PATH}

# --- CONFIGURE GIT USER HERE ---
print("Configuring Git user...")
!git config user.name "{GITHUB_USERNAME}"
!git config user.email "{GITHUB_USERNAME}@users.noreply.github.com"

# print("Setup complete. You are ready to work.")

Mounted at /content/drive
Repository already exists. Syncing with GitHub...
/content/PyNucleus-Model
Already up to date.
/content/PyNucleus-Model
Configuring Git user...


In [4]:
# --- Installation Cell (Run this ONCE per session) ---
print("Installing all required packages...")

# Core LangChain and Community Packages
!pip install -q langchain langchain-core langchain-community langchain-text-splitters

# Document Loading & Processing (Unstructured handles many file types including OCR)
!pip install -q "unstructured[local-inference]"

# LLM & ML Libraries
!pip install -q transformers accelerate bitsandbytes torch sentence-transformers

# Vector Stores
!pip install -q chromadb faiss-cpu # faiss-gpu if you have a Pro Colab with a good GPU

# Data Handling & Utilities
!pip install -q pandas numpy tqdm PyYAML

print("All packages installed successfully.")

Installing all required packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/5

In [5]:
# --- Core Utilities & File Handling ---
import getpass
import PyYAML
import yaml # Make sure PyYAML is installed
from pathlib import Path

# --- Data Handling & Progress Bars ---
import pandas as pd
import numpy as np
from tqdm import tqdm

# --- Document Loading ---
# UnstructuredFileLoader is now in langchain_community
from langchain_community.document_loaders import UnstructuredFileLoader
# For future testing of more specific loaders:
# from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader

# --- OCR Libraries ---
# No direct imports needed here if using UnstructuredFileLoader with local-inference,
# as it handles OCR internally.
# You would only import these for a manual OCR process:
# import pytesseract
# from pdf2image import convert_from_path
# from PIL import Image

# --- Core ML & LLM Libraries ---
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes
import accelerate # Often used with transformers

# --- Vector Stores & Embeddings ---
# Using ChromaDB as the primary vector store.
import chromadb
from langchain_community.vectorstores import Chroma # Chroma is now in langchain_community
from langchain_community.embeddings import HuggingFaceEmbeddings # Embeddings are also in langchain_community
# For future testing of a high-performance alternative:
# import faiss
# from langchain_community.vectorstores import FAISS

# --- RAG/Agent Frameworks ---
# Core LangChain components
import langchain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# For future testing of an alternative RAG-focused toolkit:
# from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

# **Data Ingestion and Preprocessing for RAG**

# This is the last cell of the code

In [8]:
from datetime import datetime

# --- CREATE/UPDATE THE LOG FILE ---
# This will add a new line to your log file with the current date and time.
log_message = f"Notebook saved on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
with open(log_filename, "a") as f:
    f.write(log_message + "\n")
print(f"Updated '{log_filename}'")

# --- COPY YOUR NOTEBOOK FROM DRIVE INTO THE REPO ---
print(f"Copying '{notebook_filename_in_repo}' from Google Drive...")
# Note: We use the variables defined in the first cell.
!cp "{notebook_drive_path}" "{notebook_filename_in_repo}"

# --- ADD, COMMIT, AND PUSH BOTH FILES ---
print("Staging files for commit...")
# Add BOTH the notebook and the log file to Git
!git add "{notebook_filename_in_repo}"
!git add "{log_filename}"

# Create a commit message
commit_message = f"Update project notebook and log file - {datetime.now().strftime('%Y-%m-%d')}"
print(f"Committing with message: '{commit_message}'")
!git commit -m "{commit_message}"

# Push the changes to your 'main' branch on GitHub
print("Pushing changes to GitHub...")
!git push origin main

print("SUCCESS! Your notebook and log file have been updated on GitHub.")

Updated 'update_log.txt'
Copying 'Capstone Project.ipynb' from Google Drive...
Staging files for commit...
Committing with message: 'Update project notebook and log file - 2025-06-02'
[main 09750ff] Update project notebook and log file - 2025-06-02
 2 files changed, 2 insertions(+)
 create mode 100644 Capstone Project.ipynb
Pushing changes to GitHub...
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 4.15 KiB | 4.15 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
To https://github.com/Saytor20/PyNucleus-Model.git
   f12cb3a..09750ff  main -> main
SUCCESS! Your notebook and log file have been updated on GitHub.
